# Airbus Ship Detection

EE 435: Deep Learning Foundations from Scratch \
Northwestern Univertiy Winter 2020

## Memebers
* Blaine Rothrock (`BRY4768`)
* Ilan Ponsky (`IPW1530`)
* Will Dong (`WDG4518`)

## Code

This document is intenially missing components of our implementation and is intended as a overview what we did and learned from the project. For all notebooks and code, see our public [git repo](https://github.com/blainerothrock/airbus-ship-detection).

## Overview

Our group is interested in applying knowledge from this course to training TensorFlow models and getting a better understanding of deep artificial neural networks involved in image processing. In order to learn about this process in an organized and efficient way, we utilized a closed Kaggle competition that centered around our topic of image processing with neural networks. The competition we used was the [Airbus Ship Detection Challenge](https://www.kaggle.com/c/airbus-ship-detection). The goal of the Kaggle competition was for participants to be able to build a model that could “detect all ships in satellite images as quickly as possible.”

The specific goals we wanted to hit for our Deep Learning from Scratch final project were to: 
* Build a binary classifier model to gain a basic understanding of Tensorflow and how to build models in TensorFlow.
    - The objective of the binary classifier model is to output whether an image contained a ship or not utilizing optimization techniques in TensorFlow.
* Explore and implement a U-net model for image segmentation. 
    - U-Net models are the current state-of-art for image segmentation and where most started for this competition.
    - This is a hefty goal given the data size of ~40GB of images and the time it takes to training this complex model. Our goal is to build a model, attempt at training, and gain a understanding of the U-Net architecture.
    - To accomplish this, we utilized the notebook of Kevin Mader on Kaggle which served as an excellent foundation to get started with implementing the mentioned goals

## Understanding the Data

The main reason we choose to use Kaggle is because the data is easily obtained and well organized. The data came in the form of a set of training and testing images then a `.csv` file containing image names and encoded pixels. Each row mapped to an image and the encoded pixel column represented a bounding box around a ship in a image. Most images have no ships and some have multiple, up to 15.

First we are able to explore the segment file: `train_ship_segmentations_v2.csv`

In [1]:
import os
import pandas as pd

ship_dir = '../data/'
train_image_dir = os.path.join(ship_dir, 'train_v2')
test_image_dir = os.path.join(ship_dir, 'test_v2')

masks = pd.read_csv(os.path.join(ship_dir, 'train_ship_segmentations_v2.csv'))
print(masks.shape[0], 'masks found')
print(masks['ImageId'].value_counts().shape[0], 'unique images found')
masks.head(5)

FileNotFoundError: [Errno 2] File b'../data/train_ship_segmentations_v2.csv' does not exist: b'../data/train_ship_segmentations_v2.csv'

Using code from a getting started notebook on the kaggle project, we are able to see the masks. below we are looking at an image with the max number of ship (15). This code checks the given encoding and decoding methods

In [ ]:
import matplotlib.pyplot as plt
from util import masks_as_image
from util import multi_rle_encode

fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 5))
rle_0 = masks.query('ImageId=="11f3bae66.jpg"')['EncodedPixels'] # 15 ships in the image
img_0 = masks_as_image(rle_0)
ax1.imshow(img_0[:, :, 0])
ax1.set_title('Image$_0$')
rle_1 = multi_rle_encode(img_0)
img_1 = masks_as_image(rle_1)
ax2.imshow(img_1[:, :, 0])
ax2.set_title('Image$_1$')
print('Check Decoding->Encoding',
      'RLE_0:', len(rle_0), '->',
      'RLE_1:', len(rle_1))

## Binary Classifier

The objective of our binary classifier is to gain a understanding of building models in Tensorflow and apply optimization problems learned in this class to TensorFlow. Our Binary classifier will use the data from the Airbus challenge. Instead of using image segmentation to predict where a ship is in a image, it will simply detect if there is a ship in the image. For this, we will use a CNN model and explore training and optimization in Tensorflow.

### Adjust the data for image classification.

The first step to update the data-frame for the classification. To start we have pixel encoding and we need to translate the to classifications of `ship` and `no_ship`. When an image has multiple ships there are multiple entries in the data-frame, so they will need to be flattened as well. Below is the code we used to to that:

In [ ]:
# populate a ship count (0 or 1)
masks['ship_count'] = masks['EncodedPixels'].map(lambda x: 1 if isinstance(x, str) else 0)
masks.sample(10)

# aggregate ship images and sum number of ships. images with multiple ships will have multiple rows
unique_masks = masks.groupby('ImageId').agg({'ship_count': 'sum'}).reset_index()
unique_masks['file_size_kb'] = unique_masks['ImageId'].map(lambda c_img_id: os.stat(os.path.join(train_image_dir, c_img_id)).st_size/1024 if os.path.exists(os.path.join(train_image_dir, c_img_id)) else 0.0)
unique_masks = unique_masks[unique_masks['file_size_kb']>50]
unique_masks.drop(['file_size_kb'], axis=1, inplace=True)

# create a string class column, used later for the build in image generator
unique_masks['has_ship'] = unique_masks['ship_count'].map(lambda x: 1 if x > 0.0 else 0)
unique_masks['class'] = unique_masks['ship_count'].map(lambda x: 'ship' if x > 0.0 else 'no_ship')
unique_masks.sample(5)

## Binary Classification Training & Results

### Baseline CNN model

To start and to get familiar with TensorFlow, we started with a very simple CNN model build in TensorFlow2 (Keras). This model includes 3 hidden convolution layer with 16, 32 and 64 nodes respectively. Each convolutional layer has a max pooling layer with the deful `(2, 2)` pool sizing. Our image generator reduced the size of the ships from `(768, 768, 3)` to `(256, 256, 3`) to reduce training times. The model uses the standard Adam optimizer with default settings.

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(256, 256 ,3)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=['accuracy']
)

The results of this model gives us a baseline for optimzation. 

<img src="https://blainerothrock-public.s3.us-east-2.amazonaws.com/airbus-ship-detection/binary_classifier_results3.png" alt="initial binary classifier" style="float: left; height: 300px;"/>

The model doesn't perform horribly, but we can clearly see some issues with overfitting in the training set and the validation accuracy doesn't improve much after the first few epochs. In the next few iterations we will try and improve this by adding some common techniques to reduce overflow and produce better results.

### Iterations from the baseline

This is a consolidation to reduce the size of our report. We tired a few iterations of the following:

* **Undersamplying the data** -- reduce the training data to have a more even distribution of the binary classes.
* **Augmenting the data** -- Randomly flipping and slightly rotating training images to avoid overfitting
* **Momentum/Adam optimization** -- Adding beta values to the Adam optimizer
* **Keras Callbacks**: When the accuracy of loss of validation flat-lines, reduce the learning rate by a factor
* **Drop out/Larger Model** -- Randomly dropping nodes in the model, this again can avoid overfitting and in some cases improve the model

#### 1 . Undersampling the data
The first thing we can do is undersample the training data. We do this because the distribution is greatly unbalanced. There are many more images with no ships and therefore our model will be bias to guess no ships. Predicting no ships gives the model about 80% accuracy given the distribution:

In [ ]:
no_ship_count = unique_masks[unique_masks['class'] == 'no_ship'].shape[0]
ship_count = unique_masks[unique_masks["class"] == 'ship'].shape[0]
print('no_ship count: {}\nship count:     {}\n'.format(no_ship_count, ship_count))
no_ship_idxs = unique_masks[unique_masks['class'] == 'no_ship'].sample(no_ship_count - ship_count - 5000).index
balanced_train_df = unique_masks.drop(no_ship_idxs, inplace=False)
no_ship_count = balanced_train_df[balanced_train_df['class'] == 'no_ship'].shape[0]
print('balanced no ship count: {}'.format(no_ship_count))

### 2. Augmenting the data
Using the built in image generators provided by Tensorflow2 we can easily augment the data to avoid overfitting. We did this by adding horizontal flip and rotation to our images. We did not want to add any transformations such as stretching because of the nature of our data. The boats are very small and generally the same shape, so altering the structure of the image, we believe, would be detrimental. The following in the image generator we used to do this:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_image_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10, # rotate +/- 10 degrees
    horizontal_flip=True # flip the image
)

train_data_gen = train_image_generator.flow_from_dataframe(
    balanced_train_df,
    directory=train_image_dir,
    x_col='ImageId',
    y_col='class',
    target_size=(256, 256),
    class_mode='binary',
    shuffle=True
)

### 3. Adam Optimization
We also played around with the Adam optimizer adding beta values. We did not see any drastic changes in the first few epochs so we kept the default values for beta.

In [ ]:
optimizer = tf.keras.optimizers.Adam(
    learning_rate=10e-4, 
    beta_1=0.9, 
    beta_2=0.999, 
    epsilon=1e-07,
    name='Adam'
)

### 4. Keras Callbacks: Learning Rate Reduction on Plateau & Early Stopping
TensorFlow provides a number of built in callbacks that can be used to help train your model. We used a few to experiment including TensorBoard, Early Stopping, Reduce Learning Rate on Plateau, and Checkpoints. Early stopping is extremely helpful when training on instances that cost money (assuming you know when the model stops). Early stopping will measure a metric of the model, usually a validation metric, and stop the model if that metric doesn't improve for some number of epochs. Reduce Learning Rate on Plateau is also very useful. Instead of adjust the learning rate linearly or based on a arbitrary epoch threshold, this callback will start to reduce the learning rate when the model stops learning. These callbacks are calculated after each epoch and action are taking on training if needed. Engineers can also create custom callbacks to monitor or report on the training. 

In [ ]:
reduceLROnPlateautf = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_accuracy', 
    factor=0.1, 
    patience=3, 
    verbose=1, 
    mode='auto',
    min_delta=0, 
    cooldown=0, 
    min_lr=0
)

earlyStop = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0, 
    patience=10, 
    verbose=1, 
    mode='max',
    baseline=None, 
    restore_best_weights=False
)

### 5. Dropout & Larger Model
Applying dropout to the model with the Keras `Dropout()` Keras layer helper is a way to reduce overfitting the training data. Dropout is a technique that randomly inhibits nodes in the network 

Below is the final model we attempted, which is slightly larger than the baseline (4 convolution layers with 32, 32, 64, 64 nodes), and adds dropout at 20%.

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(256, 256 ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1)
])

### Results of optimization
We were able to reduce the about of overfitting with our data, but we were never able to get the model to learn past about the 92% validation accuracy. We trained about 10 iterations of the model, each taking about 70 - 90 minutes to complete. With this we determine that the data may not be as clean has we hoped. In a few cases we did see  images with miss-numbered boats, or that were clearly generated (repeated mirror images, etc).

Below is the learning curve from our more complex model with dropout also utilizing the learning rate reduction callback and data augmentation. The overfitting has clearly improved from the baseline, but there is no real improvement in the overall result.

<img src="images/dropout_binary.png" alt="initial binary classifier" style="float: left; height: 300px;"/>

## U-Net Model

Building upon our binary classification experiments, we wanted to implement a deep learning architecture that would actually be able to not just identify whether an image had a ship or not, but to also identify the number of ships in the image. To accomplish this, we chose to implement a U-Net architecture. 

The U-Net was first introduced in 2015 as a convolutional neural network developed for biomedical image segmentation at the Computer Science Department of the University of Freiburg, Germany. The full paper for the original U-Net can be found [here.](https://arxiv.org/abs/1505.04597) 

The reason behind choosing the U-Net architecture is partly because the U-Net is a very popular end-to-end encoder-decoder network for image segmentation. In our implementation, the U-Net detects individual objects (ships) and predicts their masks.


### U-Net Architecture

<img src="https://miro.medium.com/max/921/0*o1n7BqtOj9_xLD_x.png" alt="initial binary classifier" style="float: left; height: 300px;"/>

The U-Net model is shaped like a "U", hence its name. The model architecture is symmetric and consists of two major parts - a contracting path and an expansive path. Our implementation is given in the code chunk below, which utilizes the Keras Tensorflow library. The imported functions can be found in our utility file.

```python
from util import conv_down #These auxiliary functions are in our util file
from util import conv_up
from util import pool

d1 = conv_down(8, input_img, name='d1')
dp1 = pool(d1, name='d1')
d2 = conv_down(16, dp1, name='d2')
dp2 = pool(d2, name='d2')
d3 = conv_down(32, dp2, name='d3')
dp3 = pool(d3, name='d3')
d4 = conv_down(64, dp3, name='d4')
dp4 = pool(d4, name='d4')
b = conv_down(128, dp4, name='b')
u1 = conv_up(64, b, d4, name='u1')
u2 = conv_up(32, u1, d3, name='u2')
u3 = conv_up(16, u2, d2, name='u3')
u4 = conv_up(8, u3, d1, name='u4')

out = Conv2D(1, (1, 1), activation='sigmoid', name='out_conv1')(u4)
out = Cropping2D((unet_params.edge_crop, unet_params.edge_crop), name='out_crop')(out)
out = ZeroPadding2D((unet_params.edge_crop, unet_params.edge_crop), name='out_pad')(out)

unet = models.Model(inputs=[input_img], outputs=[out], name="UNet")
```

### Contraction

During the contraction phase, each layer typically undergoes the same path - two convolution layers, followed by a max pooling layer, and a dropout layer. The implementations we explored on Kaggle did not use a dropout layer. In our later trail we added a dropout layer to see how it performed. That trail will be discussed in the results. 

We first start on the left side of the "U", where we contract the input images. The first part of our implementation is:

```python
d1 = conv_down(8, input_img, name='d1')
dp1 = pool(d1, name='d1')
```
Where conv down and pool are:


```python
def conv_down(filter_, in_layer, name, kernel=(3, 3), activation='relu', padding='same'):
    l = Conv2D(filter_, kernel, activation=activation, padding=padding, name=name+'_conv1')(in_layer)
    l = Conv2D(filter_, kernel, activation=activation, padding=padding, name=name+'_conv2')(l)
    return l

def pool(in_layer, name, pool_size=(2, 2)):
    return MaxPooling2D(pool_size, name=name+'_pool')(in_layer)
```


Here, the "d1" variable consists of two convolutional layers (since we are doing convolutions on a 3x3 basis), which serve to increase the depth of the input image. This is followed by a max pooling process which essentially cuts the size of the image in half. In our implementation, the number of channels start at 8, and increase to 16, while the pixel size of the image goes from 586x568 to 284x284.

This process is then repeated three more times

```python
d2 = conv_down(16, dp1, name='d2')
dp2 = pool(d2, name='d2')
d3 = conv_down(32, dp2, name='d3')
dp3 = pool(d3, name='d3')
d4 = conv_down(64, dp3, name='d4')
dp4 = pool(d4, name='d4')
```
until the input image reaches the middle/lowest layer. The image is then run through two more convolutional layers, but no max pooling layer, since the next step is expansion. At this point, the image has been resized to 28x28x128, and is ready to get upsized to its original size.

### Expansion

The flow of an expansion layer consists of upsampling the input image and concatenating it with the corresponding image from the contracting path. In our implementation, it looks like this:


```python
u1 = conv_up(64, b, d4, name='u1')
u2 = conv_up(32, u1, d3, name='u2')
u3 = conv_up(16, u2, d2, name='u3')
u4 = conv_up(8, u3, d1, name='u4')
```

Where conv_up is:


```python
def conv_up(filter_, in_layer, conv_down_layer, name, upsample_size=(2, 2), kernel=(3, 3), activation='relu', padding='same'):
    l = UpSampling2D(upsample_size, name=name+'_upsample')(in_layer)
    l = concatenate([l, conv_down_layer], name=name+'_concat')
    l = Conv2D(filter_, kernel, activation=activation, padding=padding, name=name+'_conv1')(l)
    l = Conv2D(filter_, kernel, activation=activation, padding=padding, name=name+'_conv2')(l)
    return l
```

Here, we apply the UpSampling2D function from Keras, which essentially performs some padding on the original image, followed by a convolution operation (transposed convolution). After upsampling, the image size is doubled, and the image is then concatenated with the corresponding image from the corresponding contraction layer. That is, in the first layer the image is upsized from 28x28x128 to 56x56x64, and concatenated with the 56x56x64 image in the contraction layer (u1 and d4 in the first case) to make an image of size 56x56x128. By concatenating the images, information from the previous layers is combined to obtain a more precise prediciton. This process is again repeated 3 times to preserve symmetry, and trivially, an unsymmetric model would not be able to perform concatenation in the expansion phase.

## Output Layer

Once the image has gotten to the top layer in the expansion phase, the final step in the model requires us to reshape the image to satisfy the prediction requirements.

```python
out = Conv2D(1, (1, 1), activation='sigmoid', name='out_conv1')(u4)
out = Cropping2D((unet_params.edge_crop, unet_params.edge_crop), name='out_crop')(out)
out = ZeroPadding2D((unet_params.edge_crop, unet_params.edge_crop), name='out_pad')(out)
```


The output layer is a convolution layer with a single filter of size 1x1. Here, we add additional padding to fit the pixel size requirements.

### U-Net Results

To explore U-Net with our project we focused on two iterations of the baseline model found in these notebooks: 
* [Baseline U-Net Model](https://www.kaggle.com/kmader/baseline-u-net-model-part-1)
* [U-Net model with Submission](https://www.kaggle.com/hmendonca/u-net-model-with-submission)

Most of the time was spent getting acclimated with code and understand each step in the process. This problem is time consuming and complex. Looking deeper in to the code, we saw something that stuck out to us as inherently wrong with the implementation. The loss function for these notebooks yields a loss that is less than 0. This loss is defined as cross entropy minus the dive coefficient. We thought this was very odd given the fact gradient decent will be minimizing to -infinity. We tried using different loss functions that would minimize to zero, but it did not seem to make a significant difference. 

After playing around with the code for about a week, we built out own implementation of the model (below) and included dropout at each convolutional layer. We built this model using TensorFlow2 Keras, the notebooks originally were using Keras and Tensorflow1.


In [ ]:
def conv_down(filter_, in_layer, name, kernel=(3, 3), activation='relu', padding='same'):
    l = Conv2D(filter_, kernel, activation=activation, padding=padding, name=name+'_conv1')(in_layer)
    l = Conv2D(filter_, kernel, activation=activation, padding=padding, name=name+'_conv2')(l)
    return l

def pool(in_layer, name, pool_size=(2, 2)):
    return MaxPooling2D(pool_size, name=name+'_pool')(in_layer)

def conv_up(filter_, in_layer, conv_down_layer, name, upsample_size=(2, 2), kernel=(3, 3), activation='relu', padding='same'):
    l = UpSampling2D(upsample_size, name=name+'_upsample')(in_layer)
    l = concatenate([l, conv_down_layer], name=name+'_concat')
    l = Dropout(0.2)(l)
    l = Conv2D(filter_, kernel, activation=activation, padding=padding, name=name+'_conv1')(l)
    l = Conv2D(filter_, kernel, activation=activation, padding=padding, name=name+'_conv2')(l)
    return l

input_img = Input(t_x.shape[1:], name = 'RGB_Input')
input_layer = GaussianNoise(unet_params.gaussian_noise)(input_img)
input_layer = BatchNormalization()(input_layer)

d1 = conv_down(8, input_img, name='d1')
dp1 = pool(d1, name='d1')
dp1 = Dropout(0.2)(dp1)
d2 = conv_down(16, dp1, name='d2')
dp2 = pool(d2, name='d2')
dp2 = Dropout(0.2)(dp2)
d3 = conv_down(32, dp2, name='d3')
dp3 = pool(d3, name='d3')
dp3 = Dropout(0.2)(dp3)
d4 = conv_down(64, dp3, name='d4')
dp4 = pool(d4, name='d4')
dp4 = Dropout(0.2)(dp4)
b = conv_down(128, dp4, name='b')
u1 = conv_up(64, b, d4, name='u1')
u2 = conv_up(32, u1, d3, name='u2')
u3 = conv_up(16, u2, d2, name='u3')
u4 = conv_up(8, u3, d1, name='u4')

out = Conv2D(1, (1, 1), activation='sigmoid', name='out_conv1')(u4)
out = Cropping2D((unet_params.edge_crop, unet_params.edge_crop), name='out_crop')(out)
out = ZeroPadding2D((unet_params.edge_crop, unet_params.edge_crop), name='out_pad')(out)

seg_model = models.Model(inputs=[input_img], outputs=[out], name="UNet")
seg_model.summary()

After understanding the code, we made an update to this Kaggle notebook, replacing the model with our model including dropout and submitted. Our results are clearly learning, but we did not perform as well as the baseline. We believe this is because we didn't train for a long, the Kaggle notebooks are limited on time. Also these models were trained using the `v1` dataset and we used the `v2` which may have differences. Our submission can be found in in `u-net-model-with-dropout.ipynb` or on [Kaggle](https://www.kaggle.com/blainerothrock/u-net-model-with-dropout). We scored a `0.42` on our submission, which is evaluated with the following equation (F2 Score at different intersection over union):

$$IoU(A, B) = \frac{A \cap B}{A  \cup B}$$ 

The following is the learning rate for our model

<img src="images/unet-w-drop-out-learning.png" alt="initial binary classifier" style="float: left; height: 300px;"/>

And the following is how the model predicts, you can clearly see that the model is learning to detect ships:


<img src="https://blainerothrock-public.s3.us-east-2.amazonaws.com/airbus-ship-detection/preds.png" alt="initial binary classifier" style="float: left; height: 900px;"/>

## Conclusion

This was a challenge, but extremely helpful exercise that has given us some real exposure to a real deep learning problem. Our goal was to get exposure to TensorFlow and work through how to build models using the library. We believe we achieved that goal even though our results from the challenge are not state-of-the-art. This project also gave us a great introduction to Kaggle, which we will be using to gain more experience with data science and deep learning. In hindsight, we believe the following areas best outline the components we learned from this project:
* TensorFlow2/Keras model building and training
    * custom loss functions
    * Keras Callbacks (early stopping, reduce learning rate on plateau, checkpoint, TensorBoard, etc.)
* Kaggle challenges    
* CNN/U-Net Architectures

### Challenges
We faced a number of challenges while working though this project and we will highlight them here. This was above all a learning experience so we will be able to learning from these challenges as we gain more experience building machine learning models:

* **Resources: Storage, AWS**
  - Initially we wanted to try and work completely on AWS for this project. We have credits from our program, but ran into a number of issues. First, AWS education accounts have little quota limits. So every time you want to spin up a instance that cost money, you must request a quota increase which can take up to a week. We did this Sagemaker and were able to spin up some Jupyter Lab instances, but in the end we decided to use a personal machine with a GTX 1080 for most of our work.
  - The data for this project was very large (40gb), which caused a number of issues working remote or on different machines. Luckily Kaggle makes this pretty easy to download.

* **TensorFlow/Keras Versioning**
    - In the case of U-Net models, we were not able to reproduce results we saw in Kaggle notebooks remotely. We are unsure if this is a problem with incomplete data or Python dependency versioning. In the end we ran our new U-Net model on Kaggle, which was actually a decent experience and we were able to get results similar to the original notebooks. It took us a long time to realize this, but we will be using Kaggle in the future.
    - We had another big problem with TensorFlow callbacks. In some cases, on our local machine, TensorFlow would not recognize metrics that were clearly stated in the compiled model. We found some resources outlining this problem, but never got a real solution. Sometimes it worked fine and other times the model would not recognize `val_loss` or `val_accurary` as a metric. This would cause early stopping and reduce on plateau to fail after the first training epoch.